In [18]:
from langgraph.graph import StateGraph, END
from langchain_groq import ChatGroq
from typing import TypedDict, Literal
from dotenv import load_dotenv
from google import genai
from google.genai import types
import os

from yfinance import utils




import streamlit as st

from classes import *
from utils import *

load_dotenv()
MODEL = "llama-3.1-8b-instant"

llm = ChatGroq(
    temperature=0,
    model_name=MODEL,
    api_key=os.environ.get("GROQ_API_KEY"),
)

test_input = UsageClassfier(user_query="""
I'm 32 years old, working as a full-time software engineer in Bangalore. 
I make around ₹1.2 lakh per month with a side freelance income of ₹20,000. 
My investment goal is to build long-term wealth for early retirement, ideally by the age of 50. 
I’ve been investing for about 4 years, mostly in mutual funds, but I want to be more active now. 
Given that I have moderate risk tolerance and two young children, I want a strategy that balances growth and stability.
What investment duration and asset mix would you recommend?
""")

# test_input1 = usage_extractor(test_input)

# test_input2 = portfolio_builder(test_input)

# test_input3 = portfolio_summariser(test_input2)



# #App Title
# st.title("Stock Adivce/Strategy")

# # User Input Section
# st.subheader("Enter Your Query")
# user_query = st.text_input("Ask your financial question (e.g., 'Advice related to NVIDIA Stocks', 'Give me trading strategy based on my portfolio')")


# if st.button("Get Solution"):
#     if user_query:
#         state = app.invoke({"user_query"} : user_query)

In [36]:
graph = StateGraph(AppState)
graph.support_multiple_edges = True

graph.add_node("usage_extractor", usage_extractor)
graph.add_node("news_extractor", news_extractor)
graph.add_node("macro_economic", macro_economic)
graph.add_node("market_trendsX", market_trends)
graph.add_node("adviceX", advice)
graph.add_node("strategyX", strategy)
graph.add_node("final_proposalX", final_proposal)

graph.add_conditional_edges("market_trendsX", usage_check, {"advice": "adviceX", "strategy" : "strategyX", "invalid" : "final_proposalX"})

graph.add_edge("usage_extractor", "news_extractor")
graph.add_edge("usage_extractor", "macro_economic")
graph.add_edge("news_extractor", "market_trendsX")
graph.add_edge("macro_economic", "market_trendsX")
graph.add_edge("adviceX", "final_proposalX")
graph.add_edge("strategyX", "final_proposalX")

graph.set_entry_point("usage_extractor")
graph.set_finish_point("final_proposalX")
app = graph.compile()

In [39]:
png_graph = app.get_graph().draw_mermaid_png()

with open("Pipeline.png", "wb") as f:
    f.write(png_graph)


Graph saved as 'Pipeline.png' in c:\Users\Azeem\Desktop\AI Token\Stock-advice-agent


In [23]:
NASDAQ = {
    "NVDA": "Nvidia",
    "AAPL": "Apple",
    "MSFT": "Microsoft",
    "AMZN": "Amazon",
    "GOOG": "Alphabet Class C",
    "GOOGL": "Alphabet Class A",
    "META": "Meta Platforms",
    "TSLA": "Tesla",
    "AVGO": "Broadcom",
    "NFLX": "Netflix",
    "COST": "Costco",
    "ASML": "ASML Holding",
    "TMUS": "T-Mobile US",
    "CSCO": "Cisco Systems",
    "ISRG": "Intuitive Surgical",
    "AZN": "AstraZeneca",
    "LIN": "Linde",
    "PEP": "PepsiCo",
    "AMD": "Advanced Micro Devices",
    "QCOM": "Qualcomm",
    "ADBE": "Adobe",
    "TXN": "Texas Instruments",
    "INTU": "Intuit",
    "VZ": "Verizon",
    "BKNG": "Booking Holdings",
    "AMAT": "Applied Materials",
    "AMGN": "Amgen",
    "HON": "Honeywell",
    "PDD": "Pinduoduo",
    "CMCSA": "Comcast",
    "LRCX": "Lam Research",
    "LULU": "Lululemon Athletica",
    "MAR": "Marriott International",
    "MRVL": "Marvell Technology",
    "MELI": "Mercado Libre",
    "MCHP": "Microchip Technology",
    "MU": "Micron Technology",
    "MSTR": "MicroStrategy",
    "MDLZ": "Mondelez International",
    "MNST": "Monster Beverage",
    "NXPI": "NXP Semiconductors",
    "ORLY": "O’Reilly Automotive",
    "ODFL": "Old Dominion Freight Line",
    "ON": "Onsemi",
    "PCAR": "Paccar",
    "PLTR": "Palantir Technologies",
    "PANW": "Palo Alto Networks",
    "PAYX": "Paychex",
    "PYPL": "PayPal Holdings",
    "REGN": "Regeneron Pharmaceuticals"
}


In [12]:
import yfinance as yf
import pandas_ta as ta
import random
from stocks import *

In [7]:
stock = random.choice(list(NASDAQ))
stock

'NVDA'

In [1]:
stocks = ["AAPL"]

In [ ]:
"""
Indicators Calculated:
RSI (Relative Strength Index)

MACD and MACD Signal

SMA / EMA (20-day)

Bollinger Bands (Upper/Mid/Lower)

ATR (Average True Range)

OBV (On-Balance Volume)
""" 

In [63]:
df = yf.download("NVDA", period="12mo", interval="1d")
df.dropna(inplace=True)

if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)


# Now apply RSI
df["RSI"] = ta.rsi(df["Close"], length=14)

macd = ta.macd(df["Close"])
df["MACD_Line"] = macd["MACD_12_26_9"]
df["MACD_Signal"] = macd["MACDs_12_26_9"]
df["SMA_20"] = ta.sma(df["Close"], length=20)
df["EMA_20"] = ta.ema(df["Close"], length=20)


bbands = ta.bbands(df["Close"], length=20)
df["BB_upper"]  = bbands["BBU_20_2.0"]
df["BB_middle"] = bbands["BBM_20_2.0"]
df["BB_lower"]  = bbands["BBL_20_2.0"]
df["BB_bandwidth"] = bbands["BBB_20_2.0"]
df["BB_percent"]   = bbands["BBP_20_2.0"]

for idx, row in df.tail(1).iterrows():
    for col, val in row.items():
        print(f"{col}: {val}")



[*********************100%***********************]  1 of 1 completed

Close: 157.75
High: 158.7100067138672
Low: 155.25999450683594
Open: 156.0399932861328
Volume: 258753300.0
RSI: 76.61969847906771
MACD_Line: 6.247877544692187
MACD_Signal: 5.632167582449503
SMA_20: 144.5480987548828
EMA_20: 144.70259584444779
BB_upper: 155.4428272143912
BB_middle: 144.5480987548828
BB_lower: 133.6533702953744
BB_bandwidth: 15.074191294598926
BB_percent: 1.1058848228381135


In [45]:
df

Price,Close,High,Low,Open,Volume
Date,,,,,
2024-06-28,209.639572,215.064215,209.321070,214.765608,82542700
2024-07-01,215.741043,216.497500,210.933525,211.102731,60402900
2024-07-02,219.244644,219.354133,214.098712,215.143812,58046200
2024-07-03,220.518707,220.518707,218.010433,218.975919,37369800
2024-07-05,225.286407,225.395896,220.618236,220.618236,60412400
...,...,...,...,...,...
2025-06-23,201.500000,202.300003,198.960007,201.630005,55814300
2025-06-24,200.300003,203.440002,200.199997,202.589996,54064000
2025-06-25,201.559998,203.669998,200.619995,201.449997,39525700


Date
2025-06-27    49.816928
Name: RSI, dtype: float64


Close: 201.0800018310547
High: 203.22000122070312
Low: 200.0
Open: 201.88999938964844
Volume: 73114100.0
RSI: 49.81692804835132
MACD_Line: -0.73399320243999
MACD_Signal: -1.1704988325193109
SMA_20: 200.44100036621094
EMA_20: 200.67693599397091
BB_upper: 204.89602544450673
BB_middle: 200.44100036621094
BB_lower: 195.98597528791515
BB_bandwidth: 4.44522335266372
BB_percent: 0.5717169324092992


Close: 201.0800018310547
High: 203.22000122070312
Low: 200.0
Open: 201.88999938964844
Volume: 73114100.0
RSI: 49.81692804835132
MACD_Line: -0.73399320243999
MACD_Signal: -1.1704988325193109
SMA_20: 200.44100036621094
EMA_20: 200.67693599397091
BB_upper: 204.89602544450673
BB_middle: 200.44100036621094
BB_lower: 195.98597528791515
BB_bandwidth: 4.44522335266372
BB_percent: 0.5717169324092992


In [50]:
bbands = ta.bbands(df["Close"], length=20)
print(bbands.columns)

Index(['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0'], dtype='object')


In [ ]:
import requests

API_KEY = 'Q0MsaeFzc4PRLk2QhQGIUixPM0gVashK'

    


📈 Sector Performance:
Materials: -0.16605%
Communication Services: 1.36639%
Financials: -0.67036%
Health Care: -0.09209%
Industrials: 0.67094%

💰 Sector P/E Ratios:


In [5]:

BASE_STABLE = "https://financialmodelingprep.com/stable"
BASE_V3 = "https://financialmodelingprep.com/api/v3"
BASE_V4 = "https://financialmodelingprep.com/api/v4"

def fetch_json(url, label=""):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        if isinstance(data, dict) and "Error Message" in data:
            print(f"❌ API error in {label}: {data['Error Message']}")
            return None
        return data
    except requests.exceptions.HTTPError as http_err:
        print(f"❌ HTTP error in {label}: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"❌ Request error in {label}: {req_err}")
    except ValueError:
        print(f"❌ JSON decode error in {label}")
    return None

# 1. Sector P/E snapshot
sector_pe_url = f"{BASE_STABLE}/sector-pe-snapshot?apikey={API_KEY}"
sector_pe = fetch_json(sector_pe_url, "Sector P/E")

# 2. Historical Sector P/E (example: Technology)
hist_pe_url = f"{BASE_STABLE}/historical-sector-pe?sector=Technology&apikey={API_KEY}"
hist_pe = fetch_json(hist_pe_url, "Historical Sector P/E")

# 3. Real-time Sector Performance
sector_perf_url = f"{BASE_V3}/sectors-performance?apikey={API_KEY}"
sector_perf = fetch_json(sector_perf_url, "Sector Performance")

# 4. Industry-level P/E Ratios
industry_pe_url = f"{BASE_V4}/industry_price_earning_ratio?apikey={API_KEY}"
industry_pe = fetch_json(industry_pe_url, "Industry P/E")

# 5. Company financial ratios (example: AAPL)
symbol = "AAPL"
company_ratios_url = f"{BASE_V3}/ratios/{symbol}?period=ttm&apikey={API_KEY}"
company_ratios = fetch_json(company_ratios_url, f"{symbol} Ratios")

# -------------------------
# Display Results
# -------------------------
print("\n📊 Sector P/E Snapshot:")
if sector_pe:
    for d in sector_pe:
        print(f"{d.get('sector', 'N/A')}: P/E = {d.get('peRatio', 'N/A')}")

print("\n📈 Sector Performance:")
if sector_perf:
    for d in sector_perf:
        print(f"{d.get('sector', 'N/A')}: Change = {d.get('changesPercentage', 'N/A')}")

print("\n🧱 Industry P/E:")
if industry_pe:
    for d in industry_pe:
        print(f"{d.get('industry', 'N/A')}: P/E = {d.get('peRatio', 'N/A')}")

print(f"\n📌 {symbol} Key Financial Ratios:")
if company_ratios:
    latest = company_ratios[0] if len(company_ratios) > 0 else {}
    print(f"ROE: {latest.get('roe', 'N/A')}, P/B: {latest.get('priceBookRatio', 'N/A')}, P/E: {latest.get('priceEarningsRatio', 'N/A')}")


❌ HTTP error in Sector P/E: 400 Client Error: Bad Request for url: https://financialmodelingprep.com/stable/sector-pe-snapshot?apikey=Q0MsaeFzc4PRLk2QhQGIUixPM0gVashK
❌ HTTP error in Industry P/E: 403 Client Error: Forbidden for url: https://financialmodelingprep.com/api/v4/industry_price_earning_ratio?apikey=Q0MsaeFzc4PRLk2QhQGIUixPM0gVashK

📊 Sector P/E Snapshot:

📈 Sector Performance:
Basic Materials: Change = -0.16605%
Communication Services: Change = 1.36639%
Consumer Cyclical: Change = 0.86569%
Consumer Defensive: Change = 0.3637%
Energy: Change = -0.77668%
Financial Services: Change = -0.67036%
Healthcare: Change = -0.09209%
Industrials: Change = 0.67094%
Real Estate: Change = 1.74762%
Technology: Change = -0.22351%
Utilities: Change = -1.43448%

🧱 Industry P/E:

📌 AAPL Key Financial Ratios:
ROE: N/A, P/B: N/A, P/E: 37.287278415656736


In [1]:
import websocket
import json
import threading
import time
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("FINNHUB_API")

def on_message(ws, message):
    data = json.loads(message)
    if 'data' in data:
        for item in data['data']:
            print(f"[{time.strftime('%H:%M:%S')}] {item['s']} → Price: {item['p']}")

def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status_code, close_msg):
    print("WebSocket closed")

def on_open(ws):
    # Subscribe to AAPL price feed
    ws.send(json.dumps({"type": "subscribe", "symbol": "AAPL"}))

# Run WebSocket in a thread
def run_websocket():
    url = f"wss://ws.finnhub.io?token={API_KEY}"
    ws = websocket.WebSocketApp(
        url,
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
    ws.run_forever()

# Run for 1 minute
ws_thread = threading.Thread(target=run_websocket)
ws_thread.start()
time.sleep(60)
print("Stopping feed after 60 seconds")


[19:23:54] AAPL → Price: 200.88
[19:23:58] AAPL → Price: 200.86
[19:24:04] AAPL → Price: 200.9
[19:24:06] AAPL → Price: 200.91
[19:24:10] AAPL → Price: 200.91
[19:24:14] AAPL → Price: 200.88
[19:24:18] AAPL → Price: 200.84
[19:24:29] AAPL → Price: 200.84
[19:24:29] AAPL → Price: 200.84
[19:24:35] AAPL → Price: 200.87
[19:24:35] AAPL → Price: 200.88
[19:24:35] AAPL → Price: 200.88
[19:24:36] AAPL → Price: 200.9
[19:24:37] AAPL → Price: 200.91
[19:24:37] AAPL → Price: 200.91
[19:24:37] AAPL → Price: 200.91
[19:24:37] AAPL → Price: 200.91
[19:24:37] AAPL → Price: 200.91
[19:24:37] AAPL → Price: 200.91
[19:24:37] AAPL → Price: 200.91
[19:24:37] AAPL → Price: 200.91
[19:24:37] AAPL → Price: 200.91
[19:24:40] AAPL → Price: 200.87
[19:24:40] AAPL → Price: 200.89
[19:24:42] AAPL → Price: 200.88
[19:24:42] AAPL → Price: 200.87
Stopping feed after 60 seconds


[19:24:55] AAPL → Price: 200.84
[19:24:55] AAPL → Price: 200.83
[19:24:58] AAPL → Price: 200.83
[19:25:03] AAPL → Price: 200.84
[19:25:03] AAPL → Price: 200.84
[19:25:03] AAPL → Price: 200.83
[19:25:03] AAPL → Price: 200.83
[19:25:07] AAPL → Price: 200.82
[19:25:07] AAPL → Price: 200.8
[19:25:07] AAPL → Price: 200.8
[19:25:10] AAPL → Price: 200.83
[19:25:10] AAPL → Price: 200.83
[19:25:17] AAPL → Price: 200.85
[19:25:17] AAPL → Price: 200.86
[19:25:17] AAPL → Price: 200.86
[19:25:19] AAPL → Price: 200.875
[19:25:20] AAPL → Price: 200.85
[19:25:20] AAPL → Price: 200.86
[19:25:20] AAPL → Price: 200.86
[19:25:23] AAPL → Price: 200.85
[19:25:25] AAPL → Price: 200.84
[19:25:27] AAPL → Price: 200.83
[19:25:27] AAPL → Price: 200.82
[19:25:29] AAPL → Price: 200.77
[19:25:29] AAPL → Price: 200.77
[19:25:29] AAPL → Price: 200.77
[19:25:29] AAPL → Price: 200.76
[19:25:29] AAPL → Price: 200.76
[19:25:29] AAPL → Price: 200.76
[19:25:29] AAPL → Price: 200.76
[19:25:29] AAPL → Price: 200.77
[19:25:29